In [ ]:
!pip install crossref
!pip install elsapy
!pip install xmltodict
!pip install tqdm

In [ ]:
#Elsevier Libraries
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

#Libraries for URL access
import time
import json
import csv
import pprint
import requests
import xmltodict
import urllib3
import numpy as np
import pandas as pd
from tqdm import tqdm
#Import warnings library
import warnings
warnings.filterwarnings("ignore")

#xml.etree.ElementTree — The ElementTree XML API
import xml.etree.ElementTree as ET

info_list = [] #Creating an empty list to store the  info
#Rerunning this will delete the data that will be stored further
#Run only once

#Create a num array of 0 to 5900 spaces 100 apart per page. Each page of Search returns 100 entries until 6000 is reached
num = np.linspace(0, 5900, 60, dtype = int)
print(num)
#Maximum of 6000 searches are returned on calling the ElsSearch API

#Append "apikey" and "insttoken" as suggest in the ElsSearch document into a config file
config = {"apikey": "Enter your API", "insttoken": "Enter the inst token"}

client = ElsClient(config['apikey'])
client.inst_token = config['insttoken']

def XML_DOI(link):

    #defining a header's dictionary to pass through requests
    headers_dict = {"X-ELS-APIKey": "2dc442325fc67f2f275ec3157ef8df65", "X-ELS-Insttoken": "6beb1f6c29d85f50029bf11c8de94d1b", "Accept": "application/xml"}

    #x takes response of the HTTP request, passes link
    x = requests.get(link, headers=headers_dict)

    #Save it as XML file
    with open("doi.xml", 'wb') as f:
        f.write(x.content)

link_list = []

#Loop to iterate over all papers
#UPI query can take many arguments.
#start, count, query are a few as name suggests
for i in range(len(num)):

    start = "https://api.elsevier.com/content/search/sciencedirect?"
    count = "start=" + str(num[i]) + "&count=100"
    query = "&query=Sustainable+Battery+Recycling"
    endapi = "&apiKey=Enter your API key&insttoken=Enter your inst key"
    link = start + count + query  + endapi
    #print(link)
    link_list.append(link)
#subscription = "&subscribed=true"

link_list[6]

# Call the XML_DOI function

for j in range(len(link_list)):
    xmlfile = XML_DOI(link_list[j])
    #Read the data
    #<?xml version="1.0" encoding="UTF-8"?>
    tree = ET.parse('/content/doi.xml')
    root = tree.getroot()
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):

        info_dict = {} #This will create a dictionary where we will store information about the searches

        '''
        CHECK CODE
        url = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text
        title = entry.find('{http://purl.org/dc/elements/1.1/}title').text
        pub_name = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text
        doi = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi').text
        #description = entry.find('{http://purl.org/dc/elements/1.1/}description').text
        print(url, title, pub_name, doi)
        print('\n')
        '''

        info_dict['URL'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text
        info_dict['Title'] = entry.find('{http://purl.org/dc/elements/1.1/}title').text
        info_dict['Pub_Name'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text
        doi = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi')
        if doi is None:
            info_dict['DOI'] = None
        else:
            info_dict['DOI'] = doi.text

        info_list.append(info_dict)

# %% [code]
type(info_list), len(info_list)

print(len(info_list))

# %% [code]
DOI =  []
Title = []
Pub_name = []

for i in range(len(info_list)):
    DOI.append(info_list[i]['DOI'])
    Title.append(info_list[i]['Title'])
    Pub_name.append(info_list[i]['Pub_Name'])

# %% [code]
df =  pd.DataFrame()
df['Title'] =  Title
df['Pub_name'] = Pub_name
df['DOI'] = DOI

df.head()

# %% [code]
#Create string1 and string2 to join doi with institoken to make a single URL
string1 = "https://api.elsevier.com/content/article/doi/"
string2 = "?apiKey=Enter your API key&insttoken=Enter your inst key"

#Access every DOI in the previous file and append the new URL to another column
df['Link'] = df['DOI'].apply(lambda x: string1 + str(x) + string2)

df.head()

list_abstract = []
list_doi= []
list_title = []
list_date = []
list_journal = []

from pprint import pprint
doi= df['DOI']
for i in range(len(info_list)):
    doi = info_list[i]['DOI']
    if doi is not None:
        doi_doc = FullDoc(doi = doi)
        if doi_doc.read(client):

            #pprint(dir(doi_doc))
            #pprint(doi_doc._data)
            abstract = doi_doc._data['coredata']['dc:description']
            title = doi_doc.title
            date = doi_doc._data['coredata']['prism:coverDisplayDate']
            journal = doi_doc._data['coredata']['prism:publicationName']
            #pprint(abstract)
            #print(i, 'done')
            #dict_abstract['DOI'] = doi[i]
            #dict_abstract['Abstract'] = abstract
            list_abstract.append(abstract)
            list_doi.append(doi)
            list_title.append(title)
            list_date.append(date)
            list_journal.append(journal)
            #doi_doc.write()
        else:
            pprint('Operation failed')
            list_abstract.append('0')
            list_doi.append('1')
            list_title.append('2')
            list_date.append('3')
            list_journal.append('4')
    else:
        list_abstract.append('0')
        list_doi.append('1')
        list_title.append('2')
        list_date.append('3')
        list_journal.append('4')

# %% [code]
df_available_abstract = pd.DataFrame()
df_available_abstract['DOI'] = list_doi
df_available_abstract['Title'] = list_title
df_available_abstract['Abstract'] = list_abstract
df_available_abstract['Date'] = list_date
df_available_abstract['Journal'] = list_journal

df_available_abstract.head(len(info_list))

In [ ]:
df_available_abstract.to_csv("Enter address")